# C - TIME SERIES SMOOTHING
**REQUIREMENT : (A)**

Can be run in parallel with (B).

This program carries out time-series gap filling and smoothing (gaussian filter) for a selection of indexes using the following function:

>**smooth_time_serie(sites, dteDebut, dteFin, dteExe)**

For each SAR, this function operates the following steps:

**1) Listing of sentinel 2 indexes for which we want to obtain smoothed profiles**

**2) Selection of index raw data in the sar_index_stats tables and outlier removal for the period between dteDebut and dteFin**

**3) Gap filling and smoothing**

**4) Smoothed time series are saved in tables**

There is one table per index per site.

Tables are named as following: (id_segment)_(index)_(dteExe)_smoothed


## Import libraries and functions

In [ ]:
import os, sys, datetime
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # Localisation of SARSAR libraries

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config
from select_sites import sites_to_process

## Definition of the **smooth_time_serie** function

In [ ]:
def smooth_time_serie(sites, dteDebut, dteFin, dteExe):
    """ This function fills the gaps and smoothes index raw profiles using a gaussian filter over a given period (dteDebut-dteFin).
        The function adds tables to the DB containing the resulting smoothed index time series.
        
            Parameters (they are automatically read in "sarsar.ini" by the function "config"):
                sites (list): list of sites (sar_id_segment) to process
                dteDebut (date): date from which we need data (YYYY-MM-DD)
                dteFin (date): date until which we need data (YYYY-MM-DD)
                dteExe: processing/execution date (YYYYMMDD)
                """
    
    ## DEFINING LIST OF INDEXES TO SMOOTH
    indices = ['BAI','BI','BI2','NDVI','SBI']
    
    
    ## CONNECTION TO DB    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    
    ##CALCULATION OF SMOOTHED INDEX PROFILES

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
#         cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats;')    
#         sites = cur.fetchall()
        for site in sites:
            #sar_id_segment = dict(site) # DEBUG : Why do we create a dictionnary?
            
            for indice in indices :
                print('> Processing smoothing for %s, %s' % (site, indice))
                # DEBUG : Why not "site" instead of "sar_id_segment['sar_id_segment']"
                # Fetching raw index values over the given period
#                raw_profile = sarsar_admin.fetch_sar_index_stats_by_sar_id(sar_id_segment['sar_id_segment'], indice, dteDebut, dteFin )
                raw_profile = sarsar_admin.fetch_sar_index_stats_by_sar_id(site, indice, dteDebut, dteFin )
                
                # Removing outliers from raw profiles
                no_outlier_profile = sarsar_admin.remove_outliers_from_stats(raw_profile)

                # Smoothing time series
                smoothed_profile = sarsar_admin.compute_smoothed_time_series_stats(no_outlier_profile)

                # transposes the two arrays into a dictionary as expected for the next loop
                smoothed_dates = smoothed_profile[0]
                smoothed_values = smoothed_profile[1]
                smoothed_profile_dico = {}
                
    ## SAVING SMOOTHED INDEX PROFILES IN TABLES
                for i in range(len(smoothed_dates)):
                    smoothed_profile_dico[smoothed_dates[i].strftime("%Y-%m-%d")] = smoothed_values[i]

                # (re)create the table "{sar_id_segment}_{index}_{dteExe}_smoothed" which should contain this data
#                table_name = '{0}_{1}_{2}_smoothed'.format(sar_id_segment['sar_id_segment'], indice, dteExe)
                table_name = '{0}_{1}_{2}_smoothed'.format(site, indice, dteExe)
                strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)  # Surround the table name to tolerate numbers at the beginning and dashes in the middle
                cur.execute(strSQL)
                conn.commit()

                strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (dte DATE, indice NUMERIC);'.format(table_name)  #, ', '.join(type_valeurs))
                cur.execute(strSQL)
                
                #Filling table
                for item in smoothed_profile_dico:
                    strSQL = 'INSERT INTO "{0}" (dte, indice) VALUES (\'{1}\', {2});'.format(table_name, item, smoothed_profile_dico[item])  # , (', '.join(champs)), str(tuple(valeurs)))
                    cur.execute(strSQL)

            # Post all modifications
            conn.commit()
            
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

## Calling functions

In [ ]:
## Call config
dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']
dteExe = dates['exe']

## Call sites_to_process
lstSARs = sites_to_process(dteDebut, dteFin)

## Call smooth_time_serie
smooth_time_serie(lstSARs, dteDebut, dteFin, dteExe)